In [7]:
import backtrader as bt
import datetime
import pyfolio as pf
import backtrader.analyzers as btanalyzers

class RSIStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):

        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # To keep track of pending orders
        self.order = None

        # To keep track of placed positions
        self.currentposition = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # # Write down: no pending order
        # self.order = None

    def strategy_blast(self, id):
        x1 = ((self.dataclose[id] + self.datalow[id]) * (self.dataopen[id] + self.datahigh[id])) / 2
        y1 = ((self.dataclose[id] + self.datahigh[id]) * (self.dataopen[id] + self.datalow[id])) / 2
        spike = (x1/y1)

        return spike

    def next(self):

        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # # Check if an order is pending ... if yes, we cannot send a 2nd one
        # if self.order:
        #     return
        
        # self.close()

        spike_1 = self.strategy_blast(-1)
        spike = self.strategy_blast(0)


        if spike_1 > 1 and spike <= 1:
            buy = True
        else:
            buy = False
        
        if spike_1 < 1 and spike >= 1:
            sell = True
        else:
            sell = False


        # Check if we are in the market
        if not self.position:
            self.close()

            # Open all Opened Positions
            if buy == True:

                # print('Buy Position Opened, %.2f'% self.dataopen[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(size=1)
                self.currentposition = "Buy"
            
            if sell == True:
                
                # print('Sell Position Opened, %.2f'% self.dataopen[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell(size=1)
                self.currentposition = "Sell"


        else:
            # Close all Opened Positions


            if self.order is not None:

                if self.currentposition == "Buy" and buy == False:
                    self.close()
                    self.currentposition = None
                    # print('Buy Position Closed, %.2f'% self.dataclose[0])

                if self.currentposition == "Sell" and sell == False:
                    self.close()
                    self.currentposition = None
                    # print('Sell Position Closed, %.2f'% self.dataclose[0])
           




cerebro = bt.Cerebro()

fromdate = datetime.datetime.strptime('2021-08-01', '%Y-%m-%d')
todate = datetime.datetime.strptime('2021-08-30', '%Y-%m-%d')

data = bt.feeds.GenericCSVData(dataname='../data/2021_15minutes.csv', dtformat=2, compression=15, timeframe=bt.TimeFrame.Minutes, fromdate=fromdate, todate=todate)

cerebro.adddata(data)

cerebro.addstrategy(RSIStrategy)

cerebro.broker.setcash(100000.0)

# cerebro.addanalyzer(bt.analyzers.PyFolio)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')



print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# cerebro.run()
results = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()




# 2021-08-01



# cerebro.plot()

Starting Portfolio Value: 100000.00
2021-08-01, Close, 41624.67
2021-08-01, SELL EXECUTED, 41631.60
2021-08-01, Close, 41686.38
2021-08-01, BUY EXECUTED, 41686.39
2021-08-01, Close, 41737.37
2021-08-01, Close, 41860.98
2021-08-01, Close, 41733.17
2021-08-01, BUY EXECUTED, 41733.17
2021-08-01, Close, 41739.79
2021-08-01, SELL EXECUTED, 41739.80
2021-08-01, Close, 41707.22
2021-08-01, BUY EXECUTED, 41707.22
2021-08-01, Close, 41634.32
2021-08-01, SELL EXECUTED, 41634.32
2021-08-01, Close, 41710.00
2021-08-01, SELL EXECUTED, 41709.99
2021-08-01, Close, 41722.28
2021-08-01, BUY EXECUTED, 41722.28
2021-08-01, Close, 41808.25
2021-08-01, Close, 41951.64
2021-08-01, Close, 42257.11
2021-08-01, Close, 42307.45
2021-08-01, Close, 42435.13
2021-08-01, Close, 42407.11
2021-08-01, BUY EXECUTED, 42407.10
2021-08-01, Close, 42294.54
2021-08-01, SELL EXECUTED, 42288.92
2021-08-01, Close, 42242.80
2021-08-01, Close, 42057.79
2021-08-01, Close, 41734.28
2021-08-01, Close, 41835.99
2021-08-01, SELL EXEC

2021-08-07, Close, 44014.95
2021-08-07, BUY EXECUTED, 44015.55
2021-08-07, Close, 44164.11
2021-08-07, SELL EXECUTED, 44167.93
2021-08-07, Close, 44223.84
2021-08-07, Close, 44410.55
2021-08-07, Close, 44572.54
2021-08-08, Close, 44620.08
2021-08-08, Close, 44332.00
2021-08-08, BUY EXECUTED, 44333.00
2021-08-08, Close, 44352.45
2021-08-08, SELL EXECUTED, 44352.46
2021-08-08, Close, 44573.83
2021-08-08, Close, 44255.36
2021-08-08, BUY EXECUTED, 44255.36
2021-08-08, Close, 44190.72
2021-08-08, SELL EXECUTED, 44190.72
2021-08-08, Close, 44013.54
2021-08-08, Close, 44102.86
2021-08-08, SELL EXECUTED, 44102.84
2021-08-08, Close, 43978.79
2021-08-08, BUY EXECUTED, 43975.91
2021-08-08, Close, 43899.99
2021-08-08, Close, 43924.28
2021-08-08, SELL EXECUTED, 43924.28
2021-08-08, Close, 43924.98
2021-08-08, BUY EXECUTED, 43924.98
2021-08-08, Close, 44183.20
2021-08-08, Close, 44309.08
2021-08-08, Close, 44214.28
2021-08-08, BUY EXECUTED, 44214.99
2021-08-08, Close, 44322.42
2021-08-08, SELL EXECU

2021-08-11, BUY EXECUTED, 46281.71
2021-08-11, Close, 46334.58
2021-08-11, Close, 46088.99
2021-08-11, BUY EXECUTED, 46089.00
2021-08-11, Close, 45735.56
2021-08-11, SELL EXECUTED, 45735.56
2021-08-11, Close, 45723.37
2021-08-11, Close, 45756.69
2021-08-11, SELL EXECUTED, 45756.68
2021-08-11, Close, 45690.01
2021-08-11, BUY EXECUTED, 45690.01
2021-08-11, Close, 45650.08
2021-08-11, Close, 45662.27
2021-08-11, SELL EXECUTED, 45660.00
2021-08-11, Close, 45511.00
2021-08-12, BUY EXECUTED, 45510.67
2021-08-12, Close, 45836.99
2021-08-12, SELL EXECUTED, 45836.99
2021-08-12, Close, 45737.00
2021-08-12, BUY EXECUTED, 45736.99
2021-08-12, Close, 45722.34
2021-08-12, Close, 45761.74
2021-08-12, SELL EXECUTED, 45761.73
2021-08-12, Close, 45819.31
2021-08-12, BUY EXECUTED, 45819.31
2021-08-12, Close, 45883.65
2021-08-12, Close, 45923.72
2021-08-12, Close, 46000.00
2021-08-12, Close, 46006.13
2021-08-12, Close, 46073.42
2021-08-12, Close, 46141.22
2021-08-12, Close, 46213.96
2021-08-12, Close, 461

2021-08-16, BUY EXECUTED, 47085.44
2021-08-16, Close, 47089.99
2021-08-16, SELL EXECUTED, 47089.99
2021-08-16, Close, 47169.98
2021-08-16, Close, 47373.78
2021-08-16, Close, 47284.65
2021-08-16, BUY EXECUTED, 47284.65
2021-08-16, Close, 47350.76
2021-08-16, SELL EXECUTED, 47350.76
2021-08-16, Close, 47421.57
2021-08-16, Close, 47565.44
2021-08-16, Close, 47507.29
2021-08-16, BUY EXECUTED, 47507.29
2021-08-16, Close, 47472.60
2021-08-16, SELL EXECUTED, 47472.60
2021-08-16, Close, 47516.95
2021-08-16, SELL EXECUTED, 47516.95
2021-08-16, Close, 47344.15
2021-08-16, BUY EXECUTED, 47344.15
2021-08-16, Close, 47335.59
2021-08-16, Close, 47261.00
2021-08-16, Close, 47304.68
2021-08-16, SELL EXECUTED, 47304.68
2021-08-16, Close, 47248.23
2021-08-16, BUY EXECUTED, 47248.22
2021-08-16, Close, 47086.10
2021-08-16, Close, 46593.16
2021-08-16, Close, 46440.28
2021-08-16, Close, 46392.77
2021-08-16, Close, 46396.50
2021-08-16, SELL EXECUTED, 46396.51
2021-08-16, Close, 46204.15
2021-08-16, BUY EXECU

2021-08-22, BUY EXECUTED, 49137.87
2021-08-22, Close, 49191.84
2021-08-22, Close, 49218.41
2021-08-22, Close, 49287.85
2021-08-22, Close, 49342.20
2021-08-22, Close, 49287.61
2021-08-22, BUY EXECUTED, 49287.62
2021-08-22, Close, 49115.05
2021-08-22, SELL EXECUTED, 49115.05
2021-08-22, Close, 49178.93
2021-08-22, SELL EXECUTED, 49178.93
2021-08-22, Close, 49159.53
2021-08-22, BUY EXECUTED, 49159.53
2021-08-22, Close, 49128.31
2021-08-22, Close, 49132.04
2021-08-22, SELL EXECUTED, 49132.04
2021-08-22, Close, 48990.78
2021-08-22, BUY EXECUTED, 48990.79
2021-08-22, Close, 48961.71
2021-08-22, Close, 48898.40
2021-08-22, Close, 48861.55
2021-08-22, Close, 48810.01
2021-08-22, Close, 48949.99
2021-08-22, SELL EXECUTED, 48949.99
2021-08-22, Close, 48780.72
2021-08-22, BUY EXECUTED, 48780.71
2021-08-22, Close, 48861.87
2021-08-22, SELL EXECUTED, 48861.88
2021-08-22, Close, 48850.44
2021-08-22, BUY EXECUTED, 48850.44
2021-08-22, Close, 48782.52
2021-08-22, Close, 48630.91
2021-08-22, Close, 482

2021-08-26, BUY EXECUTED, 46928.25
2021-08-26, Close, 47006.00
2021-08-26, Close, 46979.93
2021-08-26, BUY EXECUTED, 46977.31
2021-08-26, Close, 47112.23
2021-08-26, SELL EXECUTED, 47112.22
2021-08-26, Close, 47127.36
2021-08-26, Close, 47051.51
2021-08-26, BUY EXECUTED, 47051.51
2021-08-26, Close, 46835.89
2021-08-26, SELL EXECUTED, 46835.90
2021-08-26, Close, 46559.74
2021-08-26, Close, 47385.63
2021-08-26, SELL EXECUTED, 47385.63
2021-08-26, Close, 47165.35
2021-08-26, BUY EXECUTED, 47165.35
2021-08-26, Close, 46943.76
2021-08-26, Close, 46875.94
2021-08-26, Close, 47107.89
2021-08-26, SELL EXECUTED, 47107.90
2021-08-26, Close, 47186.04
2021-08-26, BUY EXECUTED, 47186.04
2021-08-26, Close, 47195.75
2021-08-26, Close, 47360.00
2021-08-26, Close, 47109.99
2021-08-26, BUY EXECUTED, 47109.99
2021-08-26, Close, 46930.00
2021-08-26, SELL EXECUTED, 46930.01
2021-08-26, Close, 46838.94
2021-08-26, Close, 46957.02
2021-08-26, SELL EXECUTED, 46957.02
2021-08-26, Close, 46956.58
2021-08-26, BU

In [8]:

# Analyzer
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')

thestrats = cerebro.run()
thestrat = thestrats[0]

print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis())

2021-08-01, Close, 41624.67
2021-08-01, SELL EXECUTED, 41631.60
2021-08-01, Close, 41686.38
2021-08-01, BUY EXECUTED, 41686.39
2021-08-01, Close, 41737.37
2021-08-01, Close, 41860.98
2021-08-01, Close, 41733.17
2021-08-01, BUY EXECUTED, 41733.17
2021-08-01, Close, 41739.79
2021-08-01, SELL EXECUTED, 41739.80
2021-08-01, Close, 41707.22
2021-08-01, BUY EXECUTED, 41707.22
2021-08-01, Close, 41634.32
2021-08-01, SELL EXECUTED, 41634.32
2021-08-01, Close, 41710.00
2021-08-01, SELL EXECUTED, 41709.99
2021-08-01, Close, 41722.28
2021-08-01, BUY EXECUTED, 41722.28
2021-08-01, Close, 41808.25
2021-08-01, Close, 41951.64
2021-08-01, Close, 42257.11
2021-08-01, Close, 42307.45
2021-08-01, Close, 42435.13
2021-08-01, Close, 42407.11
2021-08-01, BUY EXECUTED, 42407.10
2021-08-01, Close, 42294.54
2021-08-01, SELL EXECUTED, 42288.92
2021-08-01, Close, 42242.80
2021-08-01, Close, 42057.79
2021-08-01, Close, 41734.28
2021-08-01, Close, 41835.99
2021-08-01, SELL EXECUTED, 41840.38
2021-08-01, Close, 41

2021-08-07, BUY EXECUTED, 43705.00
2021-08-07, Close, 43529.71
2021-08-07, BUY EXECUTED, 43529.71
2021-08-07, Close, 43470.30
2021-08-07, SELL EXECUTED, 43470.30
2021-08-07, Close, 43378.85
2021-08-07, Close, 43424.69
2021-08-07, SELL EXECUTED, 43428.72
2021-08-07, Close, 43319.00
2021-08-07, BUY EXECUTED, 43318.99
2021-08-07, Close, 43516.53
2021-08-07, SELL EXECUTED, 43516.52
2021-08-07, Close, 43568.78
2021-08-07, BUY EXECUTED, 43568.78
2021-08-07, Close, 43542.00
2021-08-07, BUY EXECUTED, 43542.00
2021-08-07, Close, 43644.89
2021-08-07, SELL EXECUTED, 43644.89
2021-08-07, Close, 43650.10
2021-08-07, Close, 43515.94
2021-08-07, BUY EXECUTED, 43515.95
2021-08-07, Close, 43931.41
2021-08-07, SELL EXECUTED, 43931.41
2021-08-07, Close, 43701.28
2021-08-07, BUY EXECUTED, 43700.10
2021-08-07, Close, 43922.24
2021-08-07, SELL EXECUTED, 43922.24
2021-08-07, Close, 43941.61
2021-08-07, Close, 43931.07
2021-08-07, BUY EXECUTED, 43931.07
2021-08-07, Close, 44146.41
2021-08-07, SELL EXECUTED, 4

2021-08-11, Close, 46188.87
2021-08-11, Close, 46160.37
2021-08-11, Close, 46194.32
2021-08-11, SELL EXECUTED, 46194.33
2021-08-11, Close, 46064.63
2021-08-11, BUY EXECUTED, 46064.63
2021-08-11, Close, 46221.08
2021-08-11, SELL EXECUTED, 46221.07
2021-08-11, Close, 46247.15
2021-08-11, BUY EXECUTED, 46246.09
2021-08-11, Close, 46108.58
2021-08-11, BUY EXECUTED, 46108.59
2021-08-11, Close, 46186.19
2021-08-11, SELL EXECUTED, 46186.18
2021-08-11, Close, 46013.56
2021-08-11, BUY EXECUTED, 46010.56
2021-08-11, Close, 46073.61
2021-08-11, SELL EXECUTED, 46073.60
2021-08-11, Close, 46042.05
2021-08-11, BUY EXECUTED, 46042.05
2021-08-11, Close, 46134.60
2021-08-11, SELL EXECUTED, 46134.61
2021-08-11, Close, 46139.90
2021-08-11, Close, 46048.97
2021-08-11, BUY EXECUTED, 46048.97
2021-08-11, Close, 45924.08
2021-08-11, SELL EXECUTED, 45922.33
2021-08-11, Close, 45930.00
2021-08-11, SELL EXECUTED, 45930.00
2021-08-11, Close, 45892.99
2021-08-11, BUY EXECUTED, 45892.99
2021-08-11, Close, 46017.56

2021-08-17, BUY EXECUTED, 46845.15
2021-08-17, Close, 46927.99
2021-08-17, SELL EXECUTED, 46928.00
2021-08-17, Close, 47013.08
2021-08-17, Close, 46790.15
2021-08-17, BUY EXECUTED, 46790.16
2021-08-17, Close, 46655.24
2021-08-17, SELL EXECUTED, 46655.23
2021-08-17, Close, 46410.94
2021-08-17, Close, 46522.75
2021-08-17, SELL EXECUTED, 46522.76
2021-08-17, Close, 46564.41
2021-08-17, BUY EXECUTED, 46564.42
2021-08-17, Close, 46373.51
2021-08-17, BUY EXECUTED, 46373.51
2021-08-17, Close, 46156.41
2021-08-17, SELL EXECUTED, 46156.40
2021-08-17, Close, 45916.05
2021-08-17, Close, 45964.72
2021-08-17, SELL EXECUTED, 45964.71
2021-08-17, Close, 45948.44
2021-08-17, BUY EXECUTED, 45948.45
2021-08-17, Close, 45894.79
2021-08-17, Close, 45914.61
2021-08-17, SELL EXECUTED, 45914.62
2021-08-17, Close, 45697.67
2021-08-17, BUY EXECUTED, 45697.66
2021-08-17, Close, 45730.00
2021-08-17, SELL EXECUTED, 45730.00
2021-08-17, Close, 45878.20
2021-08-17, BUY EXECUTED, 45878.20
2021-08-17, Close, 45863.00

2021-08-23, SELL EXECUTED, 49378.04
2021-08-23, Close, 49301.27
2021-08-23, BUY EXECUTED, 49301.27
2021-08-23, Close, 49177.70
2021-08-23, Close, 49418.77
2021-08-23, SELL EXECUTED, 49418.77
2021-08-23, Close, 49363.13
2021-08-23, BUY EXECUTED, 49363.14
2021-08-23, Close, 49664.14
2021-08-23, SELL EXECUTED, 49664.14
2021-08-23, Close, 49854.99
2021-08-23, BUY EXECUTED, 49854.99
2021-08-23, Close, 49822.89
2021-08-23, BUY EXECUTED, 49822.89
2021-08-23, Close, 49770.28
2021-08-23, SELL EXECUTED, 49770.29
2021-08-23, Close, 49785.00
2021-08-23, SELL EXECUTED, 49785.00
2021-08-23, Close, 49826.16
2021-08-23, BUY EXECUTED, 49826.16
2021-08-23, Close, 50046.27
2021-08-23, Close, 50112.24
2021-08-23, Close, 49891.45
2021-08-23, BUY EXECUTED, 49891.44
2021-08-23, Close, 50275.30
2021-08-23, SELL EXECUTED, 50273.56
2021-08-23, Close, 50260.00
2021-08-23, BUY EXECUTED, 50260.00
2021-08-23, Close, 50209.51
2021-08-23, SELL EXECUTED, 50209.50
2021-08-23, Close, 50159.72
2021-08-23, Close, 50270.00

2021-08-27, Close, 48226.04
2021-08-27, Close, 48207.18
2021-08-27, BUY EXECUTED, 48207.20
2021-08-27, Close, 48168.58
2021-08-27, SELL EXECUTED, 48168.58
2021-08-27, Close, 48249.24
2021-08-27, SELL EXECUTED, 48249.23
2021-08-27, Close, 48269.32
2021-08-27, BUY EXECUTED, 48269.32
2021-08-27, Close, 48300.00
2021-08-27, Close, 48325.48
2021-08-27, Close, 48389.59
2021-08-27, Close, 48343.57
2021-08-27, BUY EXECUTED, 48343.57
2021-08-27, Close, 48350.01
2021-08-27, SELL EXECUTED, 48350.00
2021-08-27, Close, 48293.69
2021-08-27, BUY EXECUTED, 48293.68
2021-08-27, Close, 48416.53
2021-08-27, SELL EXECUTED, 48416.54
2021-08-27, Close, 48806.76
2021-08-27, Close, 48960.00
2021-08-27, Close, 48895.24
2021-08-27, BUY EXECUTED, 48895.24
2021-08-27, Close, 48986.08
2021-08-27, SELL EXECUTED, 48986.07
2021-08-27, Close, 48955.95
2021-08-27, BUY EXECUTED, 48955.96
2021-08-27, Close, 49037.59
2021-08-27, SELL EXECUTED, 49037.58
2021-08-27, Close, 48960.39
2021-08-27, BUY EXECUTED, 48960.40
2021-08